In [1]:
import mxnet as mx
import sagemaker
from sagemaker.mxnet import MXNet as MXNetEstimator

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
mx.test_utils.get_cifar10() # Downloads Cifar-10 dataset to ./data/cifar

In [3]:
inputs = sagemaker_session.upload_data(path='data/cifar', key_prefix='data/cifar10')

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-968277166688


In [4]:
estimator = MXNetEstimator(entry_point='train.py', 
            role=sagemaker.get_execution_role(), 
            train_instance_count=1, 
            train_instance_type='ml.p3.2xlarge',
            hyperparameters={'batch_size': 1024, 
                             'epochs': 30})
estimator.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-968277166688
INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-06-29-00-39-51-275


.......................................
2018-06-29 00:46:12,095 INFO - root - running container entrypoint
2018-06-29 00:46:12,095 INFO - root - starting train task
2018-06-29 00:46:12,115 INFO - container_support.training - Training starting
2018-06-29 00:46:16,016 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 1, 'channels': {u'training': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'network_interface_name': u'ethwe', u'hosts': [u'algo-1']}, 'user_script_name': u'train.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'training': u'/opt/ml/input/data/training'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/output', 'model_dir': '/opt/ml/model', 'hyperparameters': {u'sagemaker_program': u'train.py', u'batch_size': 1024, u'epochs

2018-06-29 00:47:54,335 INFO - root - [Epoch 24] training: accuracy=0.951471
2018-06-29 00:47:54,335 INFO - root - [Epoch 24] time cost: 3.232138
2018-06-29 00:47:54,662 INFO - root - [Epoch 24] validation: accuracy=0.715169
2018-06-29 00:47:57,880 INFO - root - [Epoch 25] training: accuracy=0.958426
2018-06-29 00:47:57,880 INFO - root - [Epoch 25] time cost: 3.218427
2018-06-29 00:47:58,238 INFO - root - [Epoch 25] validation: accuracy=0.728613
2018-06-29 00:48:01,465 INFO - root - [Epoch 26] training: accuracy=0.964565
2018-06-29 00:48:01,465 INFO - root - [Epoch 26] time cost: 3.227052
2018-06-29 00:48:01,826 INFO - root - [Epoch 26] validation: accuracy=0.742578
2018-06-29 00:48:05,056 INFO - root - [Epoch 27] training: accuracy=0.964505
2018-06-29 00:48:05,056 INFO - root - [Epoch 27] time cost: 3.230222
2018-06-29 00:48:05,414 INFO - root - [Epoch 27] validation: accuracy=0.733887
2018-06-29 00:48:08,652 INFO - root - [Epoch 28] training: accuracy=0.967654
2018-06-29 00:48:08,652

In [5]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-06-29-00-39-51-275
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-06-29-00-39-51-275


---------------------------------------------------!

### Evaluate

We'll use these CIFAR-10 sample images to test the service:

<img style="display: inline; height: 32px; margin: 0.25em" src="images/airplane1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/automobile1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/bird1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/cat1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/deer1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/dog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/frog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/horse1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/ship1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/truck1.png" />



In [11]:
# load the CIFAR-10 samples, and convert them into format we can use with the prediction endpoint
from readimage_utils import read_images

filenames = ['images/airplane1.png',
             'images/automobile1.png',
             'images/bird1.png',
             'images/cat1.png',
             'images/deer1.png',
             'images/dog1.png',
             'images/frog1.png',
             'images/horse1.png',
             'images/ship1.png',
             'images/truck1.png']

image_data = read_images(filenames)

In [12]:
for i, img in enumerate(image_data):
    response = predictor.predict(img)
    print('image {}: class: {}'.format(i, int(response)))

image 0: class: 0
image 1: class: 1
image 2: class: 2
image 3: class: 4
image 4: class: 4
image 5: class: 5
image 6: class: 6
image 7: class: 7
image 8: class: 8
image 9: class: 9


In [ ]:
estimator.delete_endpoint()